In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [180]:
train_df = pd.read_csv("titanic/train.csv")
vali_df = pd.read_csv("titanic/test.csv")

In [181]:
train_df[["Pclass","Sex","Age","SibSp","Parch","Fare"]].head()

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,22.0,1,0,7.2500
1,1,female,38.0,1,0,71.2833
2,3,female,26.0,0,0,7.9250
3,1,female,35.0,1,0,53.1000
4,3,male,35.0,0,0,8.0500


In [182]:
vali_df[["Pclass","Sex","Age","SibSp","Parch","Fare"]].describe()

,Pclass,Age,SibSp,Parch,Fare
count,418.000000,332.000000,418.000000,418.000000,417.000000
mean,2.265550,30.272590,0.447368,0.392344,35.627188
std,0.841838,14.181209,0.896760,0.981429,55.907576
min,1.000000,0.170000,0.000000,0.000000,0.000000
25%,1.000000,21.000000,0.000000,0.000000,7.895800
50%,3.000000,27.000000,0.000000,0.000000,14.454200
75%,3.000000,39.000000,1.000000,0.000000,31.500000
max,3.000000,76.000000,8.000000,9.000000,512.329200


In [183]:
x = train_df.iloc[:,[2,4,5,6,7,9]].values
x_vali = vali_df.iloc[:,[1,3,4,5,6,8]].values

In [184]:
y = train_df.iloc[:,1].values

In [185]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values="NaN",strategy="mean",axis=0)
imputer = imputer.fit(x[:,2:3])
x[:,2:3] = imputer.transform(x[:,2:3])

In [186]:
imputer_x_vali = Imputer(missing_values="NaN",strategy="median",axis=0)
imputer_x_vali = imputer_x_vali.fit(x_vali[:,[2,5]])
x_vali[:,[2,5]] = imputer_x_vali.transform(x_vali[:,[2,5]])

In [187]:
# x_vali[[266,372],5:6] = 14.454200

In [188]:
from sklearn.preprocessing import LabelEncoder
labelencoder_x = LabelEncoder()
x[:,1] = labelencoder_x.fit_transform(x[:,1])
labelencoder_v = LabelEncoder()
x_vali[:,1] = labelencoder_v.fit_transform(x_vali[:,1])

In [189]:
for i in range(0,418):
    if x_vali[i,5:6] == 0:
        print(i)

266
372


In [190]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
onehotencoder_x = OneHotEncoder(categorical_features = [1])
x = onehotencoder_x.fit_transform(x).toarray()


onehotencoder_v = OneHotEncoder(categorical_features = [1])
x_vali = onehotencoder_v.fit_transform(x_vali).toarray()

In [191]:
x[0,:]

array([ 0.  ,  1.  ,  3.  , 22.  ,  1.  ,  0.  ,  7.25])

In [192]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=87)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [193]:
from sklearn.preprocessing import StandardScaler
standarscaler_x = StandardScaler()
x_train = standarscaler_x.fit_transform(x_train)
x_test = standarscaler_x.transform(x_test)
standarscaler_x_vali = StandardScaler()
x_vali = standarscaler_x_vali.fit_transform(x_vali)

In [39]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=87)
model.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=87, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [150]:
from sklearn.svm import SVC
model = SVC(kernel='rbf',random_state=87)
model.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=87, shrinking=True,
  tol=0.001, verbose=False)

In [211]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=87)
model.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=87, verbose=0, warm_start=False)

In [27]:
import keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=4,kernel_initializer="uniform",activation="relu",input_dim=7))
model.add(Dense(units=4,kernel_initializer="uniform",activation="relu"))
model.add(Dense(units=1,kernel_initializer="uniform",activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(x_train,y_train,batch_size=10,epochs=100)

C:\Users\Administrator\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Epoch 1/100
712/712 [==============================] - 2s 3ms/step - loss: 0.6909 - acc: 0.5871
Epoch 2/100
712/712 [==============================] - 0s 646us/step - loss: 0.6779 - acc: 0.6152
Epoch 3/100
712/712 [==============================] - 0s 399us/step - loss: 0.6395 - acc: 0.6685
Epoch 4/100
712/712 [==============================] - 0s 414us/step - loss: 0.5789 - acc: 0.7865
Epoch 5/100
712/712 [==============================] - 0s 397us/step - loss: 0.5281 - acc: 0.7767
Epoch 6/100
712/712 [==============================] - 0s 317us/step - loss: 0.4996 - acc: 0.7795
Epoch 7/100
712/712 [==============================] - 0s 353us/step - loss: 0.4840 - acc: 0.7795
Epoch 8/100
712/712 [==============================] - 0s 277us/step - loss: 0.4753 - acc: 0.7795
Epoch 9/100
712/712 [==============================] - 0s 235us/step - loss: 0.4692 - acc: 0.7795
Epoch 10/100
712/712 [==============================] - 0s 219us/step - loss: 0.4655 - acc: 0.7809
Epoch 11/100
712/712 

In [213]:
y_pred = model.predict(x_test)

In [214]:
y_pred = y_pred>0.5

In [215]:
acc_model = round(model.score(x_test, y_test) * 100, 2)

In [216]:
acc_model

80.45

In [217]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [218]:
cm

array([[91, 20],
       [15, 53]], dtype=int64)

In [201]:
(98+51)/179

0.8324022346368715

In [202]:
y_vali = model.predict(x_vali)

In [203]:
y_vali

array([0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [205]:
df = pd.DataFrame(y_vali)
df.to_csv("file_path1.csv",)

In [206]:
x_vali_id = vali_df.iloc[:,[0]].values

In [210]:
df = pd.DataFrame({"PassengerId" : x_vali_id, "Survived" : y_vali})
df.to_csv("submission.csv", index=False)

In [208]:

x_vali_id = vali_df.iloc[:,0].values

In [209]:
x_vali_id

array([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,  902,
        903,  904,  905,  906,  907,  908,  909,  910,  911,  912,  913,
        914,  915,  916,  917,  918,  919,  920,  921,  922,  923,  924,
        925,  926,  927,  928,  929,  930,  931,  932,  933,  934,  935,
        936,  937,  938,  939,  940,  941,  942,  943,  944,  945,  946,
        947,  948,  949,  950,  951,  952,  953,  954,  955,  956,  957,
        958,  959,  960,  961,  962,  963,  964,  965,  966,  967,  968,
        969,  970,  971,  972,  973,  974,  975,  976,  977,  978,  979,
        980,  981,  982,  983,  984,  985,  986,  987,  988,  989,  990,
        991,  992,  993,  994,  995,  996,  997,  998,  999, 1000, 1001,
       1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012,
       1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023,
       1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034,
       1035, 1036, 1037, 1038, 1039, 1040, 1041, 10